In [24]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [27]:
import pickle
import pandas as pd

In [28]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [29]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [30]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### What's the standard deviation of the predicted duration for this dataset?

In [9]:
import numpy as np
np.std(y_pred)

5.28140357655334

In [21]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [22]:
df_result = pd.DataFrame(df['ride_id'])
df_result['predictions'] = y_pred

In [23]:
output_file = 'output_file'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
df_result.set_index('ride_id')

,predictions
ride_id,
2022/02_0,18.527783
2022/02_1,23.065782
2022/02_2,33.686359
2022/02_3,23.757436
2022/02_4,21.492904
...,...
2022/02_2979426,12.038225
2022/02_2979427,11.441569
2022/02_2979428,11.890459
